https://www.langchain.com.cn/docs/tutorials/llm_chain/

In [10]:
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())


#### 使用语言模型

In [2]:
# ChatModel是LangChain“运行接口”的实例，这意味着它们提供了一个标准接口供我们与之交互
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL"),
    temperature=0.7
)


from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from Chinese into proper English that correspond native culture."),
    HumanMessage(content="黑色给了我黑色的眼睛, 我却用它寻找光明"),
]

response = model.invoke(messages)

#### 输出解析器

In [ ]:
# 模型的响应是一个AIMessage。这包含一个字符串响应以及关于响应的其他元数据。我们通常可能只想处理字符串响应。我们可以通过使用简单的输出解析器来解析出这个响应
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = model.invoke(messages)
parser.invoke(result)

'Black gave me black eyes, but I use them to seek the light.'

In [9]:
# 使用 LCEL 连接组件: 我们可以使用 | 运算符轻松创建链。| 运算符在 LangChain 中用于将两个元素组合在一起
chain = model | parser
chain.invoke(messages)

'Black gave me black eyes, but I use them to seek the light.'

#### 提示词模板

In [18]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {target_lang}:" 

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{translate_text}")]
)

result = prompt_template.invoke({"target_lang": "english", "translate_text": "你一会看我一会看云我觉得你看我时很远你看云时很近"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into english:', additional_kwargs={}, response_metadata={}), HumanMessage(content='你一会看我一会看云我觉得你看我时很远你看云时很近', additional_kwargs={}, response_metadata={})])

In [19]:
result.to_messages()

[SystemMessage(content='Translate the following into english:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你一会看我一会看云我觉得你看我时很远你看云时很近', additional_kwargs={}, response_metadata={})]

In [20]:
chain = prompt_template | model | parser
chain.invoke({"target_lang": "english", "translate_text": "你一会看我一会看云我觉得你看我时很远你看云时很近"})

'Sometimes you look at me, sometimes you look at the clouds. I feel that when you look at me, you seem distant, but when you look at the clouds, you seem very close.'